# Quantitative Stock Selection

In [2]:
import yfinance as yf 
import pandas as pd
from matplotlib import pylab as plt
import numpy as np
from datetime import datetime
import math
from time import sleep
import quandl
import seaborn as sns
import sys
from scipy.stats import norm

## Functions

In [2]:
# FUNCTIONS FOR VIEWING SINGLE COMPANY RANKINGS 

def company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            return data.loc[data['Company'] == i]
    print('No company found')
    
def Company(company):
    for i in data['Company']:
        if company in i or company in i.lower():
            stock = data.loc[data['Company'] == i]
            stock = stock[compact]
            return stock
    print('No company found')

## Data Import and Wrangling

In [11]:
# IMPORT LATEST DATA 

#data_tmp = pd.read_excel("../R-projects/Investment/Stocks/Data/Borsdata_2020-06-01-2.xlsx")
data_tmp = pd.read_excel("equity_data/Borsdata_2021-02-25.xlsx")
data_tmp = data_tmp[1:]

data_tmp = data_tmp.rename({'Performance' : 'Return 3m','Performance.1' : 'Return 6m',
                            'Performance.2' : 'Return 1y', 'Div. Yield': 'Yield', 'Info' : 'Country',
                            'Info.1' : 'List', 'Info.2' : 'Sector', 'Info.3' : 'Industry', 'Info.4' : 'Tick'}, axis=1)

# FORMAT FOR VIEWING PLEASURE
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

In [12]:
# MAKE NUMBERS FLOATS 

data_tmp['Market Cap'] = data_tmp['Market Cap'].replace(',', '') # remove , as 1000 separator

list = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Yield', 'Market Cap', 'Volatility',
        'F-Score', 'ROC', 'ROE', 'FCF', 'Total Equity ', 
        'Return 3m', 'Return 6m', 'Return 1y']

for i in list: 
    data_tmp[i] = pd.to_numeric(data_tmp[i]) # make every number numeric


data_tmp['FCFROE'] = data_tmp['FCF']/data_tmp['Total Equity '] # calc FCFROE
data_tmp['GPA'] = data_tmp['Gross profit']/data_tmp['Tot. Assets'] 

In [13]:
# REMOVE TOO SMALL COMPANIES AND FINANCIALS 

data_tmp = data_tmp.loc[data_tmp['Market Cap'] > 500]
data_tmp = data_tmp.loc[(data_tmp['List'] != 'First North') & (data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]



data_tmp = data_tmp.loc[(data_tmp['Sector'] != 'Financials')]
data_tmp.index = range(len(data_tmp)) # make new index after removing
data = data_tmp

In [14]:
# MANAGE NaNs and NEGATIVE EARNINGS  

# set nan to median
columns = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'ROC', 'ROE',
           'F-Score', 'GPA', 'Assets Turn']
for i in columns: 
    data.loc[data[i].isna() ,i] = data[i].median()

# set nan to min
for i in ['Return 3m', 'Return 6m', 'Return 1y']:
    data.loc[data[i].isna() ,i] = data[i].min()
    
# set negative values to max
for i in ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B']:
    data.loc[data[i] < 0 ,i] = data[i].max()
    
# set nan yield & vol to 0
data.loc[data['Yield'].isna(),'Yield'] = 0
data.loc[data['Volatility'].isna(),'Volatility'] = 0

## Rankings and Sorting

In [15]:
# RANK ON DIFFERENT METRICS 

f_score_cut_off = (pd.DataFrame(data.quantile(0.2, axis=0))).loc['F-Score', 0.2]

data['Momentum'] = (data['Return 3m'] + data['Return 6m'] + data['Return 1y'])/3

# Ranking where lower value is better
for i in ['P/E', 'P/B', 'P/S', 'P/FCF', 'EV/EBIT']:
    data[i +' Rank'] = data[i].rank()
    
# Ranking where higher value is better
for i in ['Yield', 'ROE', 'ROC', 'FCFROE', 'Momentum', 'GPA', 'Assets Turn']:
    data[i + ' Rank'] = (-data[i]).rank()
    
# Remove low F-Score from Momentum
data_mom_tmp = data.copy()
data_mom_tmp.loc[data['F-Score'] < f_score_cut_off ,'Momentum Rank'] = data_mom_tmp['Momentum Rank'].max()


# Composite ranks
data['Quality Rank'] = (data['ROE Rank'] + data['ROC Rank'] + data['FCFROE Rank'] + 
                        data['GPA Rank'] + data['Assets Turn Rank']).rank()
data['Value Rank'] = (data['P/E Rank'] + data['P/B Rank'] + data['P/S Rank'] + data['P/FCF Rank'] + 
                      data['EV/EBIT Rank'] + data['Yield Rank']).rank()
 
data['Magic Formula'] = (data['ROC Rank'] + data['EV/EBIT Rank']).rank()
data['Siegfrieds'] = (data['ROC Rank'] + data['P/B Rank']).rank()
data['Triple Sort'] = ((data['Value Rank'] + data['Momentum Rank'] + data['Quality Rank'])).rank()
data['Double Sort'] = (data['Value Rank'] + data['Quality Rank']).rank()

In [16]:
# SORTING 

# number of stocks to choose before momentum screen
trend_screen = 40
double_sort_break_off = 40

value_tmp = data.sort_values(by=['Value Rank'])
quality_tmp = data.sort_values(by=['Quality Rank'])
momentum_tmp = data_mom_tmp.sort_values(by=['Momentum Rank'])


trending_value_tmp = value_tmp[0:trend_screen]
trending_value_tmp = trending_value_tmp.sort_values(by='Momentum Rank')
trending_quality_tmp = quality_tmp[0:trend_screen]
trending_quality_tmp = trending_quality_tmp.sort_values(by='Momentum Rank')
magic_formula_tmp = data.sort_values(by=['Magic Formula'])
siegfrieds_tmp = data.sort_values(by=['Siegfrieds'])
triple_sort_tmp = data.sort_values(by=['Triple Sort'])
double_sort = data.sort_values(by=['Double Sort'])
double_sort_mom_tmp = double_sort[0:double_sort_break_off]
double_sort_mom = double_sort_mom_tmp.sort_values(by=['Momentum Rank'])

In [20]:
# FACTOR STRATEGIES 

compact = ['Company', 'List', 'Quality Rank', 'Value Rank', 'Momentum Rank', 'Tick']

compact_mom = ['Company','Tick', 'List', 'Momentum Rank', 'F-Score' ]

# numbers of stocks to select for diff strategies
stocks = 20
magic_formula_stocks = 10
siegfried_stocks = 10
triple_sort_stocks = 20
double_sort_mom_stocks = 10

## VALUE ## 
value = value_tmp[0:stocks]
Value = value[compact]

## QUALITY ## 
quality = quality_tmp[0:stocks]
Quality = quality[compact]

## MOMENTUM ##
momentum = momentum_tmp[0:stocks]
Momentum = momentum[compact_mom]

## TRENDING VALUE ##
trending_value = trending_value_tmp[0:stocks]
Trending_Value = trending_value[compact]

## TRENDING QUALITY ##
trending_quality = trending_quality_tmp[0:stocks]
Trending_Quality = trending_quality[compact]

## MAGIC FORMULA ##
magic_formula = magic_formula_tmp[0:magic_formula_stocks]
Magic_Formula = magic_formula[compact]

## SIEGFRIEDS ##
siegfrieds = siegfrieds_tmp[0:siegfried_stocks]
Siegfrieds = siegfrieds[compact]

## TRIPLE SORT ##
triple_sort = triple_sort_tmp[0:triple_sort_stocks]
Triple_Sort = triple_sort[compact]

## DOUBLE SORT + Momentum ##
double_sort_mom = double_sort_mom[0:double_sort_mom_stocks]
Double_Sort_Mom = double_sort_mom[compact]

In [21]:
Triple_Sort 

,Company,List,Quality Rank,Value Rank,Momentum Rank,Tick
121,Kindred,Large Cap,6.00,62.00,8.00,KIND SDB
26,Byggmax,Mid Cap,13.50,2.00,61.00,BMAX
16,Bergs Timber,Small Cap,31.00,1.00,47.00,BRG B
88,Prevas,Small Cap,4.00,12.50,73.00,PREV B
212,Ferronordic,Mid Cap,10.00,9.00,79.00,FNM
36,Elanders,Mid Cap,76.50,18.00,6.00,ELAN B
90,Proact IT,Mid Cap,20.00,54.00,27.00,PACT
149,Arctic Paper,Small Cap,57.00,7.00,48.00,ARP
174,Nilörngruppen,Small Cap,17.00,24.50,75.00,NIL B
108,Softronic,Small Cap,12.00,40.00,72.00,SOF B


In [22]:
Momentum

,Company,Tick,List,Momentum Rank,F-Score
151,G5 Entertainment,G5EN,Mid Cap,2.00,9.00
228,Gaming Innovation,GIGSEK,Small Cap,3.00,4.00
175,Sinch,SINCH,Large Cap,5.00,5.00
36,Elanders,ELAN B,Mid Cap,6.00,9.00
198,Mips,MIPS,Mid Cap,7.00,4.00
121,Kindred,KIND SDB,Large Cap,8.00,9.00
173,Evolution Gaming,EVO,Large Cap,9.00,6.00
204,Boozt,BOOZT,Mid Cap,11.00,7.00
111,Studsvik,SVIK,Small Cap,12.00,7.00
47,Fingerprint Cards,FING B,Mid Cap,13.00,5.00


## Market Cap test

In [6]:
# IMPORT LATEST DATA 

#data_tmp = pd.read_excel("../R-projects/Investment/Stocks/Data/Borsdata_2020-06-01-2.xlsx")
data_tmp = pd.read_excel("equity_data/Borsdata_2021-02-25.xlsx")
data_tmp = data_tmp[1:]

data_tmp = data_tmp.rename({'Performance' : 'Return 3m','Performance.1' : 'Return 6m',
                            'Performance.2' : 'Return 1y', 'Div. Yield': 'Yield', 'Info' : 'Country',
                            'Info.1' : 'List', 'Info.2' : 'Sector', 'Info.3' : 'Industry', 'Info.4' : 'Tick'}, axis=1)

# FORMAT FOR VIEWING PLEASURE
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.options.display.float_format = "{:,.2f}".format

data_tmp['Market Cap'] = data_tmp['Market Cap'].replace(',', '') # remove , as 1000 separator


list = ['P/E', 'EV/EBIT', 'P/FCF', 'P/S', 'P/B', 'Assets Turn', 'Gross profit', 'Tot. Assets', 'ROC', 'Yield', 'Market Cap', 'Volatility',
        'F-Score', 'ROC', 'ROE', 'FCF', 'Total Equity ', 
        'Return 3m', 'Return 6m', 'Return 1y']

for i in list: 
    data_tmp[i] = pd.to_numeric(data_tmp[i]) # make every number numeric


sthlm_tmp = data_tmp.loc[(data_tmp['List'] != 'First North') & (data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]
full_tmp = data_tmp.loc[(data_tmp['List'] != 'Spotlight') 
                        & (data_tmp['List'] != 'NGM')]


In [11]:
(pd.DataFrame(sthlm_tmp.quantile(0.2, axis=0))).loc['Market Cap', 0.2]

1174.80069

In [16]:
(pd.DataFrame(full_tmp.quantile(0.5, axis=0))).loc['Market Cap', 0.5]

1178.74175

In [17]:
len(full_tmp)

675

In [18]:
len(sthlm_tmp)

341